In [13]:
# Dependencies
import numpy as np
import pandas as pd

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, MetaData, Table, func

In [14]:
engine = create_engine("sqlite:///hawaii.sqlite")

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [15]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [17]:
inspector = inspect(engine)

In [18]:
columns = inspector.get_columns('measurement')
for column in columns:
    print(column["name"], column["type"])

id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT


In [19]:
columns = inspector.get_columns('station')
for column in columns:
    print(column["name"], column["type"])

id INTEGER
station TEXT
name TEXT
latitude FLOAT
longitude FLOAT
elevation FLOAT


## Precipitation in June and December

In [20]:
# 1. Import the sqlalchemy extract function.
from sqlalchemy import extract

# 2. Write a query that filters the Measurement table to retrieve the 
# precipitation for the month of June. 
june_results = []
june_results = session.query(Measurement.date, Measurement.prcp).filter(extract('month', Measurement.date) == 6).all()

In [21]:
june_prcp = list(np.ravel(june_results))

In [22]:
june_prcp_df = pd.DataFrame(june_results, columns = ['date', 'tobs'])
june_prcp_df.set_index(june_prcp_df['date'], inplace=True)

In [24]:
june_prcp_df.describe()

,tobs
count,1574.000000
mean,0.136360
std,0.335731
min,0.000000
25%,0.000000
50%,0.020000
75%,0.120000
max,4.430000


In [27]:
# 2. Write a query that filters the Measurement table to retrieve the 
# precipitation for the month of Dec. 
dec_results = []
dec_results = session.query(Measurement.date, Measurement.prcp).filter(extract('month', Measurement.date) == 12).all()

In [28]:
dec_prcp = list(np.ravel(dec_results))

In [29]:
dec_prcp_df = pd.DataFrame(dec_results, columns = ['date', 'tobs'])
dec_prcp_df.set_index(dec_prcp_df['date'], inplace=True)

In [30]:
dec_prcp_df.describe()

,tobs
count,1405.000000
mean,0.216819
std,0.541399
min,0.000000
25%,0.000000
50%,0.030000
75%,0.150000
max,6.420000


## Temperature by weather station

In [42]:
# 1. Import the sqlalchemy extract function.
from sqlalchemy import extract

# 2. Write a query that filters the Measurement table to retrieve the 
# temperatures for the month of June. 
june_results = []
june_results = session.query(Measurement.date, Measurement.tobs, Measurement.station).\
        group_by(Measurement.station).\
        filter(extract('month', Measurement.date) == 6).\
        group_by(Measurement.station).all()

In [43]:
june_temp = list(np.ravel(june_results))

In [44]:
june_temp_df = pd.DataFrame(june_results, columns = ['date', 'tobs', 'station'])
june_temp_df.set_index(june_temp_df['station'], inplace=True)

In [45]:
june_temp_df

,date,tobs,station
station,,,
USC00511918,2010-06-01,74.0,USC00511918
USC00513117,2010-06-01,74.0,USC00513117
USC00514830,2010-06-01,73.0,USC00514830
USC00516128,2010-06-01,70.0,USC00516128
USC00517948,2010-06-01,77.0,USC00517948
USC00518838,2010-06-01,69.0,USC00518838
USC00519281,2010-06-01,71.0,USC00519281
USC00519397,2010-06-01,78.0,USC00519397
USC00519523,2010-06-01,76.0,USC00519523
